In [4]:
include("../src/DSI.jl")
include("../src/Zono_utils.jl")
include("../src/PZono.jl")
include("../src/DSZ.jl")
include("../src/marcelstuff.jl")

pbox_from_data (generic function with 1 method)

In [11]:
using SpecialFunctions

# P-box setup

In [52]:
#b = makepbox(interval(0,1))
using PyPlot
using Distributions

c = normal(interval(0,3),1)
ProbabilityBoundsAnalysis.plot(c)

arr = []
for mean in 0:0.2:3
    normal_dist = Normal(mean, 1)
    push!(arr, normal_dist)
    x = range(mean - 4, mean + 4, length=1000)
    cdf_values = cdf.(normal_dist, x)
    plot(x, cdf_values)
end

PyPlot.savefig("parametric.png")

In [61]:
# Toy example: defining the network
W1 = [1 -1.0; 1.0 1.]
b = [0.0; 0.0]
W2 = [1 -1.0; 1.0 1.]
L1 = Layer(W1, b, ReLU())
L2 = Layer(W2, b, Id())
full_net = Network([L1; L2])

# input range
x = [normal(interval(0,3),1), normal(interval(0,3),1)]

2-element Vector{pbox}:
 Pbox: 	  ~ normal ( range=[-3.0902, 6.0902], mean=[0.0, 3.0], var=1.0)
 Pbox: 	  ~ normal ( range=[-3.0902, 6.0902], mean=[0.0, 3.0], var=1.0)

# Moment-propagation approach

In [99]:
function propagate_network(mean_x, cov_x, network::Network)
    for layer in network.layers
        mean_x, cov_x = propagate_layer(mean_x, cov_x, layer)
    end
    return mean_x, cov_x
end

propagate_network (generic function with 1 method)

In [126]:
function propagate_layer(mean_x, cov_x, layer::Layer)
    mean_y = layer.weights * mean_x .+ layer.bias
    cov_y = layer.weights * cov_x * transpose(layer.weights)
    if layer.activation === ReLU()
        mean_z, cov_z = propagate_relu(mean_y, cov_y)
    elseif layer.activation === Id()
        mean_z, cov_z = mean_y, cov_y
    end
    
    return mean_z, cov_z
end

propagate_layer (generic function with 1 method)

In [98]:
function propagate_relu(mean_y, cov_y)
    var_y = diag(cov_y)
    std_y = sqrt.(var_y)

    std_y = std_y .+ 1e-8

    alpha = mean_y ./ std_y
    CDF_alpha = 0.5 * (1 .+ erf.(alpha ./ sqrt(2))) 
    PDF_alpha = exp.(-0.5 .* alpha.^2) ./ sqrt(2π) 

    # Compute new mean
    mean_z = mean_y .* CDF_alpha .+ std_y .* PDF_alpha

    derelu = []
    push!(derelu,CDF_alpha)
    push!(derelu, PDF_alpha ./ std_y)
    push!(derelu, -mean_y ./ (std_y.^3) .* PDF_alpha)
    push!(derelu, PDF_alpha ./ (std_y.^3) .* ((mean_y.^2 ./ (std_y.^2)) .- 1))

    n = length(mean_y)
    cov_z = zeros(n, n)
    for i in 1:n
        for j in 1:n
            rho_ij = cov_y[i, j] / (std_y[i] * std_y[j])
            for k in 1:4
                cov_z[i, j] += ((rho_ij^k)/factorial(k)) * (std_y[i]^k * derelu[k][i]) * (std_y[j]^k * derelu[k][j])
            end
        end
    end

    return mean_z, cov_z
end

propagate_relu (generic function with 1 method)

Check if this works on a simple example:

In [120]:
close("all")

In [ ]:
mean_example = [1.0, 2.0]
cov_example = [1.0 0; 0 1.0]

mean_z, cov_z = propagate_network(mean_example, cov_example, full_net)
normal_dist = Normal(mean_z[2], cov_z[2,2])
supp = range(mean_z[1] - 20, mean_z[1] + 20, length=1000)
cdf_values = cdf.(normal_dist, supp)
plot(supp, cdf_values)
PyPlot.savefig("debug.png")

[2.0 0.0; 0.0 2.0]relu
[2.1686020890977873 -1.7094173839757847; -1.7094173839757847 2.1686020890977873]id
[2.1686020890977873 -1.7094173839757847; -1.7094173839757847 2.1686020890977873]

Bigger example:

In [127]:
nb_discretization_steps = 100
ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
pz = pbox_approximate_nnet(full_net, x, true)
ProbabilityBoundsAnalysis.plot(pz[1])

mean_arr = [0:0.01:3;]

for i in 1:length(mean_arr)
    for j in i:length(mean_arr)
        mean_input = [mean_arr[i], mean_arr[j]]
        cov_input = [1.0 0; 0 1.0]
        mean_output, cov_output = propagate_network(mean_input, cov_input, full_net)
        normal_dist = Normal(mean_output[1], cov_output[1,1])
        supp = range(mean_output[1] - 10, mean_output[1] + 10, length=1000)
        cdf_values = cdf.(normal_dist, supp)
        plot(supp, cdf_values)
    end
end
PyPlot.savefig("MomentPropagationfst.png")


In [128]:
nb_discretization_steps = 100
ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
pz = pbox_approximate_nnet(full_net, x, true)
ProbabilityBoundsAnalysis.plot(pz[2])

mean_arr = [0:0.01:3;]

for i in 1:length(mean_arr)
    for j in i:length(mean_arr)
        mean_input = [mean_arr[i], mean_arr[j]]
        cov_input = [1.0 0; 0 1.0]
        mean_output, cov_output = propagate_network(mean_input, cov_input, full_net)
        normal_dist = Normal(mean_output[2], cov_output[2,2])
        supp = range(mean_output[2] - 20, mean_output[2] + 20, length=1000)
        cdf_values = cdf.(normal_dist, supp)
        plot(supp, cdf_values)
    end
end
PyPlot.savefig("MomentPropagationsnd.png")


# Sampling approach

In [18]:
#for each distr, propagate it in the network i guess???
res = []

for d in arr
    fst = Base.rand(d, 100)
    snd = sample_from_pbox(x[2], 100)
    res_d = []
    for i in 1:length(fst)
        out = compute_output(full_net, [fst[i],snd[i]])
        push!(res_d, out)
    end
    push!(res, res_d)
end

fst_prop = [[res[i][j][1] for j in 1:length(res[1])] for i in 1:length(res)]

16-element Vector{Vector{Float64}}:
 [-0.704850469244972, 0.0, -2.4364946723471728, 0.0, -0.15126078539420315, -1.911172033189572, -3.299010175783317, -1.105826920653525, 0.0, 0.0  …  -0.18125288121293875, 0.0, -1.1514851412183935, -2.1954535696832163, 0.0, -0.4009798797881765, -1.6608224715534254, -0.33104602870842104, -6.080485547470349, -2.483785604406066]
 [-0.6145219938503121, -1.5392197520002395, 0.02490959992769118, -2.6520518064050362, -2.7893587815585494, -2.764611957309827, -0.04164439906929074, -4.171567045400095, 0.8029664820612508, -2.2161997788120913  …  0.0, 0.24633793505276702, -1.6364061159552932, -1.5861278130329333, -1.0869432827758359, -1.951386125633751, -2.556632329492825, -3.652741625043333, -0.5683618633077809, -1.6993503753462011]
 [-3.6196035158110136, -0.2626280193457562, -0.9376442206019847, -4.331980781860386, -0.9525013205577243, -1.837616864979533, -4.629165750484377, -1.941468736819099, -1.090269619824893, -2.0561280423891994  …  -3.8439531194372134, -5.

In [21]:
nb_discretization_steps = 100
ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
pz = pbox_approximate_nnet(full_net, x, true)
ProbabilityBoundsAnalysis.plot(pz[1])
for d in 1:length(arr)
    sorted_data, cdf = empirical_cdf(fst_prop[d])
    plot(sorted_data, cdf, marker="o", linestyle="-", label="Empirical CDF")
end
PyPlot.savefig("SamplingAppr1.png")

More thorough: we pick one distribution and another and we sample from the two.

In [25]:
aux = []

#samples
for d in arr
    smp = Base.rand(d,1000)
    push!(aux, smp)
end

#propagate
res = []
for i in 1:length(aux)
    for j in i:length(aux)
        out = []
        for k in 1:length(aux[i])
            out1 = compute_output(full_net, [aux[i][k],aux[j][k]])
            push!(out, out1)
        push!(res, out)
        end
    end
end

#this is super slow for some reason
fst_prop = [[res[i][j][1] for j in 1:length(res[1])] for i in 1:length(res)]
snd_prop = [[res[i][j][2] for j in 1:length(res[1])] for i in 1:length(res)]

136000-element Vector{Vector{Float64}}:
 [0.0, 1.665281141309483, 0.0, 0.0, 0.0, 0.8998710683359412, 0.9159205615940035, 1.2886027509400297, 0.0, 0.0  …  1.5754957942798244, 0.9240929626983267, 3.215107819663979, 0.0, 0.0, 2.882263458738508, 0.9803004161528321, 0.0, 0.0, 0.0]
 [0.0, 1.665281141309483, 0.0, 0.0, 0.0, 0.8998710683359412, 0.9159205615940035, 1.2886027509400297, 0.0, 0.0  …  1.5754957942798244, 0.9240929626983267, 3.215107819663979, 0.0, 0.0, 2.882263458738508, 0.9803004161528321, 0.0, 0.0, 0.0]
 [0.0, 1.665281141309483, 0.0, 0.0, 0.0, 0.8998710683359412, 0.9159205615940035, 1.2886027509400297, 0.0, 0.0  …  1.5754957942798244, 0.9240929626983267, 3.215107819663979, 0.0, 0.0, 2.882263458738508, 0.9803004161528321, 0.0, 0.0, 0.0]
 [0.0, 1.665281141309483, 0.0, 0.0, 0.0, 0.8998710683359412, 0.9159205615940035, 1.2886027509400297, 0.0, 0.0  …  1.5754957942798244, 0.9240929626983267, 3.215107819663979, 0.0, 0.0, 2.882263458738508, 0.9803004161528321, 0.0, 0.0, 0.0]
 [0.0, 1.665

In [26]:
nb_discretization_steps = 100
ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
pz = pbox_approximate_nnet(full_net, x, true)

ProbabilityBoundsAnalysis.plot(pz[1])
for ind in 1:length(fst_prop)
    sorted_data, cdf = empirical_cdf(fst_prop[ind])
    plot(sorted_data, cdf, linestyle="-", label="Empirical CDF")
end
PyPlot.savefig("SamplingAppr2fst.png")

In [27]:
nb_discretization_steps = 100
ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
pz = pbox_approximate_nnet(full_net, x, true)
ProbabilityBoundsAnalysis.plot(pz[2])
for ind in 1:length(snd_prop)
    sorted_data, cdf = empirical_cdf(snd_prop[ind])
    plot(sorted_data, cdf, linestyle="-", label="Empirical CDF")
end
PyPlot.savefig("SamplingAppr2snd.png")

P-box covering attempts

In [ ]:
ProbabilityBoundsAnalysis.plot(c)
for i in 1:10
    alpha = i / 10
    beta = 20
    midc = sample_from_pboxes_beta_param([c], 1000, alpha, beta)
    sorted_data, cdf = empirical_cdf(midc)
    plot(sorted_data, cdf, linestyle="-")
end
PyPlot.savefig("PboxSamplingBeta.png")